In [5]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds



In [6]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.2
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [8]:
train_data , validation_data , test_data = tfds.load(name = 'imdb_reviews' ,
                                                     split = ('train[:60%]' , 'train[60%:]' , 'test') 
                                                     , as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNDHV0U/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNDHV0U/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [11]:
train_sample , train_sample_label = next(iter(traibn_data.batch(10)))

In [17]:
print(train_sample[0].numpy())
print(train_sample_label[0].numpy())

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0


## Build the **model**

In [21]:
embedding = 'https://tfhub.dev/google/nnlm-en-dim50/2'

hub_layer = hub.KerasLayer(embedding , input_shape = [] , dtype = tf.string , trainable = True)

In [23]:
hub_layer(train_sample[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [27]:
model = tf.keras.Sequential([hub_layer , 
                            tf.keras.layers.Dense(16  , activation = tf.keras.activations.relu),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(1)])



In [32]:
model.compile(optimizer=tf.keras.optimizers.Adam() ,
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
               , metrics = ['accuracy'])

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 16)                816       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [34]:
history =model.fit(  train_data.shuffle(10000).batch(512) , 
                   validation_data = validation_data.batch(512) ,
                   verbose = 1 ,
                   epochs =10  )



Epoch 1/10
30/30 [==============================] - 21s 636ms/step - loss: 0.6734 - accuracy: 0.5351 - val_loss: 0.6249 - val_accuracy: 0.5915
Epoch 2/10
30/30 [==============================] - 19s 619ms/step - loss: 0.5754 - accuracy: 0.6673 - val_loss: 0.5131 - val_accuracy: 0.7427
Epoch 3/10
30/30 [==============================] - 19s 621ms/step - loss: 0.4487 - accuracy: 0.7953 - val_loss: 0.4197 - val_accuracy: 0.8172
Epoch 4/10
30/30 [==============================] - 19s 631ms/step - loss: 0.3429 - accuracy: 0.8642 - val_loss: 0.3596 - val_accuracy: 0.8409
Epoch 5/10
30/30 [==============================] - 20s 649ms/step - loss: 0.2630 - accuracy: 0.9036 - val_loss: 0.3270 - val_accuracy: 0.8544
Epoch 6/10
30/30 [==============================] - 19s 625ms/step - loss: 0.2003 - accuracy: 0.9295 - val_loss: 0.3137 - val_accuracy: 0.8672
Epoch 7/10
30/30 [==============================] - 19s 627ms/step - loss: 0.1507 - accuracy: 0.9538 - val_loss: 0.3074 - val_accuracy: 0.8658

In [38]:
_ = model.evaluate(test_data.batch(512) , verbose = 2)


49/49 - 5s - loss: 0.3675 - accuracy: 0.8479 - 5s/epoch - 102ms/step
